# Importações e Instalações de Biblioteca



In [1]:
!pip install 'aif360[Reductions,inFairness]'
!pip install xlrd


Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [2]:
from BibliotecasEFuncoes import * 

2024-05-21 14:25:38.684261: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 14:25:41.918643: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tuao/anaconda3/envs/aif360/lib/python3.11/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


# Lendo e processando a base de dados



Descrição dos dados no link: https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset




In [3]:
base = 'taiwan'
# Carregar o conjunto de dados Default of Credit Card Clients


# Carregar o conjunto de dados, ignorando as primeiras linhas que são metadados
taiwan = pd.read_csv('Datasets/UCI_Credit_Card.csv')

#Renomear a primeira coluna para "ID" e apaga-la
taiwan.rename(columns={'ID': 'Client_ID'}, inplace=True)
taiwan.rename(columns={'default.payment.next.month': 'class'}, inplace=True)
taiwan = taiwan.drop('Client_ID', axis=1)
taiwan.head()


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,class
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


**Produzindo categorias para faixa etária**

In [4]:
taiwan.loc[taiwan['AGE']<=24, 'faixa_etaria'] = 'jovem'
taiwan.loc[taiwan['AGE']>24, 'faixa_etaria'] = 'adulto'

**Produzindo Categoria Binária de sexo**

In [5]:

taiwan.loc[taiwan['SEX'].isin([1]), 'Sexo'] = 'male'
taiwan.loc[taiwan['SEX'].isin([2]), 'Sexo'] = 'female'

In [6]:
taiwan.dtypes

LIMIT_BAL       float64
SEX               int64
EDUCATION         int64
MARRIAGE          int64
AGE               int64
PAY_0             int64
PAY_2             int64
PAY_3             int64
PAY_4             int64
PAY_5             int64
PAY_6             int64
BILL_AMT1       float64
BILL_AMT2       float64
BILL_AMT3       float64
BILL_AMT4       float64
BILL_AMT5       float64
BILL_AMT6       float64
PAY_AMT1        float64
PAY_AMT2        float64
PAY_AMT3        float64
PAY_AMT4        float64
PAY_AMT5        float64
PAY_AMT6        float64
class             int64
faixa_etaria     object
Sexo             object
dtype: object

In [7]:
#separando rótulo (variável dependente y)
taiwan_y = taiwan['class']

## Codificando categorias

In [8]:
categorical_features = [i for i in taiwan.columns if taiwan.dtypes[i] in ['object', 'category']]
print(categorical_features)

['faixa_etaria', 'Sexo']


In [9]:
categorical_names = {}
encoders = {}

for feature in categorical_features:
  le = LabelEncoder()
  le.fit(taiwan[feature])

  taiwan[feature] = le.transform(taiwan[feature])

  categorical_names[feature] = le.classes_
  encoders[feature] = le

In [10]:
print(categorical_names)

{'faixa_etaria': array(['adulto', 'jovem'], dtype=object), 'Sexo': array(['female', 'male'], dtype=object)}


In [11]:
# Definindo as populações privilegiadas e não-privilegiadas
# Sexo
## Recuperando o indice
privileged_sex   = np.where(categorical_names['Sexo'] == 'male')[0]
unprivileged_sex = np.where(categorical_names['Sexo'] == 'female')[0]
## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
privileged_groups   = [{'Sexo' : privileged_sex}]
unprivileged_groups = [{'Sexo' : unprivileged_sex}]

# Idade
## Recuperando o indice
privileged_faixa_etaria   = np.where(categorical_names['faixa_etaria'] == 'adulto')[0]
unprivileged_faixa_etaria = np.where(categorical_names['faixa_etaria'] == 'jovem')[0]
## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
privileged_groups_fe   = [{'faixa_etaria' : privileged_faixa_etaria}]
unprivileged_groups_fe = [{'faixa_etaria' : unprivileged_faixa_etaria}]


# Dados privilegiados
privileged_classes = [privileged_sex, privileged_faixa_etaria]

In [12]:
taiwan = taiwan.astype(int)
taiwan.dtypes
# todos devem estar codificados em int64

LIMIT_BAL       int64
SEX             int64
EDUCATION       int64
MARRIAGE        int64
AGE             int64
PAY_0           int64
PAY_2           int64
PAY_3           int64
PAY_4           int64
PAY_5           int64
PAY_6           int64
BILL_AMT1       int64
BILL_AMT2       int64
BILL_AMT3       int64
BILL_AMT4       int64
BILL_AMT5       int64
BILL_AMT6       int64
PAY_AMT1        int64
PAY_AMT2        int64
PAY_AMT3        int64
PAY_AMT4        int64
PAY_AMT5        int64
PAY_AMT6        int64
class           int64
faixa_etaria    int64
Sexo            int64
dtype: object

In [13]:
# Definições sobre os dados para avaliação de justiça
taiwan.at[1, 'class'] = 0
label_name ='class'
protected_attribute_names = ['Sexo', 'faixa_etaria']
favorable_classes = np.where(taiwan['class'] == 1)[0]

# Preparando experimentos

In [14]:
list_gbt_params, list_rf_params = preparando_experimentos()
print(list_gbt_params)

Total de 5 conjuntos de parâmetros.
Total de 2 conjuntos de parâmetros.
[{'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.1, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.2, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.3, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.4, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.5, 'max_depth': 2, 'random_state': 2024}]


# Toda a base

## GBT

In [2]:
exp1_all_gbt = justica_exps(base, 'toda-a-base', taiwan, list_gbt_params, 'gbt', categorical_features, 'class', favorable_classes, protected_attribute_names, privileged_classes, privileged_groups, unprivileged_groups, privileged_groups_fe, unprivileged_groups_fe)

NameError: name 'justica_exps' is not defined

In [1]:
csv_result = pd.DataFrame(exp1_all_gbt,index=[5])
csv_result.to_csv('Resultados/05-Taiwan_GBT.csv')
print(csv_result)

NameError: name 'pd' is not defined

## RF


In [18]:
exp2_all_rf = justica_exps(base, 'toda-a-base', taiwan, list_rf_params, 'rf', categorical_features, 'class', favorable_classes, protected_attribute_names, privileged_classes, unprivileged_groups, privileged_groups, unprivileged_groups_fe, privileged_groups_fe)

<class 'dict'>
{'n_estimators': 100, 'min_samples_split': 50, 'criterion': 'gini', 'max_depth': 2, 'random_state': 2024}
0.80 de acuracia com desvio padrão de 0.00
0.57 de f1 com desvio padrão de 0.01
0.70 de precisão com desvio padrão de 0.04
0.80 de score no teste
STATISTICAL PARITY DIFFERENCE
Atributo sexo: 0.015303597236245525
Atributo faixa_etaria: -0.006192435476325087

DISPARATE IMPACT
Atributo sexo: 1.016253538377432
Atributo faixa_etaria: 0.9934907570156541
 
AVERAGE ODDS DIFFERENCE
Atributo sexo: 0.014295159233726817
Atributo faixa_etaria: 0.009436016248463408
 
EQUAL OPPORTUNITY DIFFERENCE
Atributo sexo: 0.007817184096182372
Atributo faixa_etaria: 0.0010284007188688937
{'index': 0, 'base': 'taiwan', 'attr_set': 'toda-a-base', 'modelo': 'rf', 'param_set': "{'n_estimators': 100, 'min_samples_split': 50, 'criterion': 'gini', 'max_depth': 2, 'random_state': 2024}", '10fold-acuracia': 0.8023809523809524, '10fold-f1': 0.5733501507147633, '10fold-precisao': 0.703087196423897, 'test

In [22]:
csv_result2 = pd.DataFrame(exp2_all_rf, index=[10])
csv_result2.to_csv('Resultados/10-Taiwan_RF.csv')
print(csv_result2)

    index    base     attr_set modelo                                          param_set  10fold-acuracia  10fold-f1  10fold-precisao  test-score  spd_sexo  spd_faixa_etaria   di_sexo  di_faixa_etaria  aod_sexo  aod_faixa_etaria  eod_sexo  eod_faixa_etaria
10      1  taiwan  toda-a-base     rf  {'n_estimators': 100, 'min_samples_split': 50,...         0.803429    0.58987          0.70483    0.797667  0.012708         -0.014685  1.013649          0.98438  0.010939         -0.005146  0.005683         -0.001543
